# Homework 1, Jorge Sanchez, February 14th 2022



##Problem 1

In [67]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

Pytorch was initialized

In [68]:
from torchvision import models

import predefined models

In [69]:
from google.colab import files
import cv2
import matplotlib.pyplot as plt

In [70]:
uploaded = files.upload()

Saving blue.jpg to blue (1).jpg
Saving green.jpg to green (1).jpg
Saving red.jpg to red (1).jpg


Uploaded images on the notebook using google colab functions

In [71]:
from PIL import Image
import imageio
%matplotlib inline
im1=Image.open('blue.jpg')
im2=Image.open('green.jpg')
im3=Image.open('red.jpg')
im_arr1=imageio.imread('blue.jpg')
im_arr2=imageio.imread('green.jpg')
im_arr3=imageio.imread('red.jpg')

Images assigned to variables for processing

In [72]:
im_arr1.shape

(747, 1000, 3)

In [73]:
im_arr2.shape

(416, 625, 3)

In [74]:
im_arr3.shape

(612, 612, 3)

Convert images to NumPy array like objects with three dimension

In [75]:
img1 = torch.from_numpy(im_arr1)
out1 = img1.permute(2, 0, 1)
img2 = torch.from_numpy(im_arr2)
out2 = img2.permute(2, 0, 1)
img3 = torch.from_numpy(im_arr3)
out3 = img3.permute(2, 0, 1)
img1.shape, out1.shape

(torch.Size([747, 1000, 3]), torch.Size([3, 747, 1000]))

Change the layout of the dimensions to be CXHXW

In [76]:

batch1=out1.float()
batch2=out2.float()
batch3=out3.float()

Convert batches to floating point

In [77]:
batch1= batch1/255
mean1=batch1.mean(-3)

batch2= batch2/255
mean2=batch2.mean(-3)

batch3= batch3/255
mean3=batch3.mean(-3)

mean1.shape, mean2.shape, mean3.shape

(torch.Size([747, 1000]), torch.Size([416, 625]), torch.Size([612, 612]))

# Problem 2

In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

In [3]:
t_c = [0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)


Create data tensors

In [4]:
def model(t_u, w1, w2,  b):
  return w2 * t_u ** 2 + w1 * t_u + b

Write out model as a function

In [5]:
def loss_fn(t_p, t_c):
  squared_diffs= (t_p - t_c)**2
  return squared_diffs.mean()

Define the loss

In [7]:
w1 = torch.ones(())
w2 = torch.ones(())
b = torch.zeros(())
t_p = model(t_u, w1,w2, b)
t_p

tensor([1310.1901, 3180.7100, 3445.4399, 6789.5103, 3225.9900, 2440.1101,
        1183.1101,  497.0399, 2390.9600, 3708.5601, 4746.9600])

Initialize parameters and invoke the model

In [8]:
loss = loss_fn(t_p, t_c)
loss

tensor(11709471.)

Check the value of the loss

In [12]:
delta = 0.1

loss_rate_of_change_w = \
    (loss_fn(model(t_u, w1 + delta, w2+delta,  b), t_c) - 
     loss_fn(model(t_u, w1 - delta,w2 - delta, b), t_c)) / (2.0 * delta)

Calculate rate of change for loss

In [13]:
learning_rate = 1e-2

w1 = w1 - learning_rate * loss_rate_of_change_w
w2 = w2 - learning_rate * loss_rate_of_change_w

Calculate scaling factor

In [14]:
loss_rate_of_change_b = \
    (loss_fn(model(t_u, w1, w2, b + delta), t_c) - 
     loss_fn(model(t_u, w1, w2, b - delta), t_c)) / (2.0 * delta)

b = b - learning_rate * loss_rate_of_change_b

Represent Gradient Descent

In [15]:
def dloss_fn(t_p, t_c):
    dsq_diffs = 2 * (t_p - t_c) / t_p.size(0)  # <1>
    return dsq_diffs

Compute derivative of loss

In [16]:
def dmodel_dw(t_u, w1, w2, b):
    return t_u

In [17]:
def dmodel_db(t_u, w1, w2, b):
    return 1.0

Apply derivatives to the model

In [18]:
def grad_fn(t_u, t_c, t_p, w1, w2, b):
    dloss_dtp = dloss_fn(t_p, t_c)
    dloss_dw = dloss_dtp * dmodel_dw(t_u, w1, w2, b)
    dloss_db = dloss_dtp * dmodel_db(t_u, w1, w2, b)
    return torch.stack([dloss_dw.sum(), dloss_db.sum()])  # <1>

function returning the gradient of the loss with respect to w and b

In [20]:
def training_loop(n_epochs, learning_rate, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        w1, w2, b = params

        t_p = model(t_u, w1, w2, b)  # <1>
        loss = loss_fn(t_p, t_c)
        grad = grad_fn(t_u, t_c, t_p, w1, w2, b)  # <2>

        params = params - learning_rate * grad

        print('Epoch %d, Loss %f' % (epoch, float(loss))) # <3>
            
    return params

Complete training loop

In [21]:
def training_loop(n_epochs, learning_rate, params, t_u, t_c,
                  print_params=True):
    for epoch in range(1, n_epochs + 1):
        w1, w2, b = params

        t_p = model(t_u, w1, w2, b)  # <1>
        loss = loss_fn(t_p, t_c)
        grad = grad_fn(t_u, t_c, t_p, w1, w2, b)  # <2>

        params = params - learning_rate * grad

        if epoch in {1, 2, 3, 10, 11, 99, 100, 4000, 5000}:  # <3>
            print('Epoch %d, Loss %f' % (epoch, float(loss)))
            if print_params:
                print('    Params:', params)
                print('    Grad:  ', grad)
        if epoch in {4, 12, 101}:
            print('...')

        if not torch.isfinite(loss).all():
            break  # <3>
            
    return params

Invoke training loop

In [24]:
training_loop(
    n_epochs = 5000, 
    learning_rate = 1e-1, 
    params = torch.tensor([1.0, 1.0, 0.0]), 
    t_u = t_u, 
    t_c = t_c)

RuntimeError: ignored